In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
allData = pd.read_csv('./AllData.csv')

In [3]:
allData.head()

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,...,PFE,PG,TRV,UNH,UTX,VZ,V,WMT,DIS,S&P500
0,6.09,6.30,12.70,1.13,20.44,-3.08,8.63,-3.46,11.15,-1.20,...,-1.11,-5.50,-1.47,2.19,7.20,-6.13,-0.87,2.68,3.73,4.36
1,1.03,5.48,18.83,1.04,4.66,5.82,1.22,3.44,-0.06,3.30,...,-1.31,7.27,-0.57,7.65,7.05,1.19,15.62,-3.72,7.94,4.06
2,1.84,9.40,10.53,-0.77,-6.73,-1.75,6.39,5.95,4.02,0.27,...,7.20,-0.61,2.12,5.72,-1.11,0.31,1.41,3.59,4.26,3.13
3,0.17,4.06,-2.59,3.27,-3.52,-0.61,-4.68,3.11,1.05,-0.45,...,1.15,-5.31,8.65,-4.73,-1.57,5.62,4.24,-3.74,-1.53,-0.75
4,-5.54,-7.27,-1.08,-9.36,-14.74,-7.74,-19.00,-2.07,-9.71,-8.93,...,-4.50,-2.12,-2.85,-0.68,-9.22,3.12,-6.34,11.73,6.03,-6.27


In [4]:
# 1 month risk-free rate
rf = 0.15

In [5]:
# Creates a copy of filtered DataFrame where we do not include the market
monthly_rf = allData.drop('S&P500', axis=1).copy() - rf

In [6]:
# Checks if S&P500 is still in there
monthly_rf.columns

Index([u'MMM', u'AXP', u'AAPL', u'BA', u'CAT', u'CVX', u'CSCO', u'KO', u'DD',
       u'XOM', u'GE', u'GS', u'HD', u'IBM', u'INTC', u'JNJ', u'JPM', u'MCD',
       u'MRK', u'MSFT', u'NKE', u'PFE', u'PG', u'TRV', u'UNH', u'UTX', u'VZ',
       u'V', u'WMT', u'DIS'],
      dtype='object')

In [7]:
# mt - rf in CAPM Formula
market_rf = allData['S&P500'] - rf

# smf.OLS doesn't provide a constant so we have to
# add our own constant
market_rf = sm.add_constant(market_rf)

# generates a dictionary of OLS Regression results
results = {x : smf.OLS(monthly_rf[x], market_rf).fit() for x in monthly_rf}

In [8]:
for x in monthly_rf:
    print('Stock: {} Alpha: {}  Beta: {}'.format(x, results[x].params[0], results[x].params[1]))


Stock: MMM Alpha: 0.33500231774  Beta: 1.05194535638
Stock: AXP Alpha: -0.256403445667  Beta: 1.20128332506
Stock: AAPL Alpha: 0.17175768356  Beta: 1.2894851519
Stock: BA Alpha: 0.370341857335  Beta: 1.05387649854
Stock: CAT Alpha: -1.04505520011  Beta: 1.36505741435
Stock: CVX Alpha: -0.861269150461  Beta: 1.19007848474
Stock: CSCO Alpha: -0.25444440453  Beta: 1.38741353502
Stock: KO Alpha: -0.346701951048  Beta: 0.648034833551
Stock: DD Alpha: -0.477687873975  Beta: 1.71023982242
Stock: XOM Alpha: -0.729006691243  Beta: 0.897274694215
Stock: GE Alpha: -0.113884504392  Beta: 1.19757841178
Stock: GS Alpha: 0.369648071868  Beta: 1.61409412072
Stock: HD Alpha: 0.972506610573  Beta: 1.09187058278
Stock: IBM Alpha: -1.02608774239  Beta: 0.964242383081
Stock: INTC Alpha: -0.214236951437  Beta: 1.06249693906
Stock: JNJ Alpha: 0.23206918076  Beta: 0.73184433329
Stock: JPM Alpha: 0.371927816538  Beta: 1.53371449908
Stock: MCD Alpha: -0.335956712153  Beta: 0.671009938539
Stock: MRK Alpha: 0.037

In [9]:
underperformers = [key for key in results.keys() if results[key].tvalues[0] < -1.697]
underperformers

['IBM']

In [10]:
overperformers = [key for key in results.keys() if results[key].tvalues[0] > 1.697]
overperformers

['V', 'HD', 'UNH']

In [44]:
market_return = allData['S&P500'].mean()

In [45]:
monthly_covariance = allData.drop('S&P500', axis=1).copy().cov().sort_index()

In [46]:
returns = {x: (rf + results[x].params[1]*(market_return - rf)) for x in results}
returns

{'AAPL': 1.2587423164404297,
 'AXP': 1.1829034456671361,
 'BA': 1.0561581426647271,
 'CAT': 1.3237218667759942,
 'CSCO': 1.3429444045299692,
 'CVX': 1.173269150460676,
 'DD': 1.6205212073078417,
 'DIS': 1.2052098344531894,
 'GE': 1.1797178377253292,
 'GS': 1.5378519281320076,
 'HD': 1.0888267227600805,
 'IBM': 0.97908774238571994,
 'INTC': 1.06357028477014,
 'JNJ': 0.77926415257382331,
 'JPM': 1.468738850128978,
 'KO': 0.70720195104849293,
 'MCD': 0.72695671215340985,
 'MMM': 1.0544976822601977,
 'MRK': 0.81212897988817823,
 'MSFT': 1.1248380860854996,
 'NKE': 0.5479358394031032,
 'PFE': 0.92821145470522026,
 'PG': 0.65555937053744961,
 'TRV': 1.0370578003253994,
 'UNH': 0.72413345410098917,
 'UTX': 1.1043843373691806,
 'V': 0.96778855167089251,
 'VZ': 0.49639786635655758,
 'WMT': 0.24323175258681712,
 'XOM': 0.9215066912425347}

In [47]:
monthly_return = allData.drop('S&P500', axis=1).copy().mean().sort_index()

In [48]:
u = np.array([1]*len(monthly_return))

In [49]:
market_weights = np.dot(np.linalg.inv(monthly_covariance),
                        returns.values() - rf*u)/(np.dot(np.dot(u.T, np.linalg.inv(monthly_covariance)),
                                                       returns.values() - rf*u))

In [50]:
market_weights

array([-1.33237521,  0.43853188,  0.36144459, -0.14138548, -0.08159769,
       -0.10732495,  0.08126039, -1.79659464,  0.11480482,  0.32298894,
        0.35700124, -0.10279051,  0.2181109 , -0.17983585,  0.28098058,
        1.19755654,  0.04448095,  1.03173043,  0.1364004 , -0.50148863,
        0.20601056, -0.5132092 , -0.07021198,  0.22027997, -0.23729501,
       -0.04082375,  0.21326513,  0.5025622 ,  0.57965148, -0.2021281 ])

In [51]:
og_market_weights = np.dot(np.linalg.inv(monthly_covariance),
                        monthly_return - rf*u)/(np.dot(np.dot(u.T, np.linalg.inv(monthly_covariance)),
                                                       monthly_return - rf*u))

In [52]:
og_market_weights

array([  1.72106128e+00,  -7.63373595e-01,  -2.07959079e-01,
         2.21690306e-04,  -1.45010246e-01,   6.75548268e-01,
        -1.92341706e-01,  -1.04763901e+00,  -1.61072218e-01,
        -2.57376562e+00,   1.05605607e+00,  -5.84316806e-02,
         7.42538739e-01,  -3.63864187e-01,  -5.50505479e-02,
         2.37711993e+00,   5.08320988e-01,  -1.35224423e+00,
         6.30582940e-01,   2.05097244e-01,   4.54979910e-01,
        -1.24382755e+00,  -8.55615394e-01,  -3.26570290e-02,
         9.25318245e-01,  -7.36934761e-01,  -2.03436205e-03,
         1.07190004e+00,  -1.31969957e-01,   5.55045842e-01])